<a href="https://colab.research.google.com/github/PlugnPlayProgramming/python/blob/main/autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Anomaly Detection by Auto Encoder

In [ ]:
# 입출력 폴더 생성
!mkdir data_input
!mkdir data_output

# 원본 데이터 파일 다운로드
!wget -O ./data_input/test_data.csv https://raw.githubusercontent.com/PlugnPlayProgramming/python/main/test_data.csv
!wget -O ./data_input/train_data.csv https://raw.githubusercontent.com/PlugnPlayProgramming/python/main/train_data.csv
!wget -O ./data_input/train_data_id.csv https://raw.githubusercontent.com/PlugnPlayProgramming/python/main/train_data_id.csv

In [1]:
# 모듈 및 전역변수
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt

TIME_STEPS = 60     # 센서 데이터의 timestamp 개수
fig_index = 1       # 차트파일 저장 색인
csv_index = 1       # CSV파일 저장 색인

**1. 공통 함수**
*   시각화 그래프를 파일로 저장
*   결과를 csv 파일로 저장





In [2]:
# 시각화 그래프를 파일로 저장
def my_plot(plt, bigsize=False) :
    global fig_index

    fig = plt.gcf()
    if not bigsize :
        fig.set_size_inches(8, 6)    # 크기(inch 단위)  640*480(in 80dpi)
    else :
        fig.set_size_inches(20, 15)  # 크기(inch 단위) 1600*1200(in 80dpi)

    plt.savefig("./data_output/Figure_" + str(fig_index).zfill(2) + ".png")
    fig_index += 1
    # plt.show()

    plt.close()

# 결과를 csv 파일로 저장
def my_export(np, fname, score):
    global csv_index
    np.savetxt("./data_output/Paper_" + str(csv_index).zfill(2) + "(" + fname + ").csv",
               score, delimiter=",", fmt="%s")
    csv_index += 1

**2. 센싱 데이터 로딩(센서 1개 1년치)**
* 테스트 데이터 : 7건
* 훈련 데이터  : 28,678건
* 훈련 데이터의 레이블(순번, 일자, 시각)

In [3]:
df = pd.read_csv('./data_input/test_data.csv')
test_sensor = np.expand_dims(df.to_numpy(), axis=2)

df = pd.read_csv('./data_input/train_data.csv')
train_sensor = np.expand_dims(df.to_numpy(), axis=2)

df = pd.read_csv('./data_input/train_data_id.csv', header=None)
train_file_id = df[0].values.tolist()

**2. 센싱 데이터 정규화 및 모델생성**


In [4]:
# 정규화 standard scalar= (x-mean)/std
training_mean = train_sensor.mean()
training_std = train_sensor.std()
df_training_value = (train_sensor - training_mean) / training_std
print("Number of training samples:", len(df_training_value))

x_train = df_training_value

Number of training samples: 28678


In [5]:
# 1D CNN기반 Autoencoder Model 생성
# Dropout Layer를 추가하여 Overfitting 방지
print("Input data shape:", np.shape(x_train))
model = keras.Sequential([
        layers.Input(shape=(x_train.shape[1], x_train.shape[2])),
        layers.Conv1D(64, 7, 1, "same", activation="relu"),
        layers.Dropout(0.2),
        layers.Conv1D(32, 7, 1, "same", activation="relu"),
        layers.Dropout(0.2),
        layers.Conv1D(16, 7, 1, "same", activation="relu"),
        layers.Dropout(0.2),
        layers.Conv1D(8, 7,  1, "same", activation="relu"),
        layers.Dropout(0.2),
        layers.Conv1D(4, 7,  1, "same", activation="relu"),
        layers.Dropout(0.2),
        layers.Conv1D(2, 7,  1, "same", activation="relu"),
        layers.Conv1DTranspose(2, 7, 1, "same", activation="relu"),
        layers.Dropout(0.2),
        layers.Conv1DTranspose(4, 7, 1, "same", activation="relu"),
        layers.Dropout(0.2),
        layers.Conv1DTranspose(8, 7, 1, "same", activation="relu"),
        layers.Dropout(0.2),
        layers.Conv1DTranspose(16, 7, 1, "same", activation="relu"),
        layers.Dropout(0.2),
        layers.Conv1DTranspose(32, 7, 1, "same", activation="relu"),
        layers.Dropout(0.2),
        layers.Conv1DTranspose(64, 7, 1, "same", activation="relu"),
        layers.Conv1DTranspose(1, 7, 1, "same"),
    ])

#  - Adam optimizer 사용, 0.001 learning rate 설정
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss="mse")
model.summary()

Input data shape: (28678, 60, 1)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 60, 64)            512       
                                                                 
 dropout (Dropout)           (None, 60, 64)            0         
                                                                 
 conv1d_1 (Conv1D)           (None, 60, 32)            14368     
                                                                 
 dropout_1 (Dropout)         (None, 60, 32)            0         
                                                                 
 conv1d_2 (Conv1D)           (None, 60, 16)            3600      
                                                                 
 dropout_2 (Dropout)         (None, 60, 16)            0         
                                                                 
 conv1d_3 (Conv1D)     

**4. 학습 실행**
* Epoch=50, batch_size=128, validation=0.2%, early_stopping은 5로 설정하여 학습
* Input과 Target을 동일하게 x_train으로 설정
* Autoencoder는 reconsturction model로 Input과 Output의 차이가 작은 방향으로 학습함



In [6]:
history = model.fit(
    x_train,
    x_train,
    epochs=50,
    batch_size=128,
    validation_split=0.2,
    callbacks=[keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, mode="min")],
)

# 학습된 모델 저장
model.save("./data_output/sensor_model.h5")


Epoch 1/50
180/180 [==============================] - 16s 14ms/step - loss: 0.1635 - val_loss: 0.0807
Epoch 2/50
180/180 [==============================] - 1s 8ms/step - loss: 0.0293 - val_loss: 0.0346
Epoch 3/50
180/180 [==============================] - 1s 8ms/step - loss: 0.0179 - val_loss: 0.0191
Epoch 4/50
180/180 [==============================] - 1s 8ms/step - loss: 0.0113 - val_loss: 0.0046
Epoch 5/50
180/180 [==============================] - 1s 8ms/step - loss: 0.0074 - val_loss: 0.0054
Epoch 6/50
180/180 [==============================] - 1s 8ms/step - loss: 0.0056 - val_loss: 0.0031
Epoch 7/50
180/180 [==============================] - 1s 8ms/step - loss: 0.0044 - val_loss: 0.0038
Epoch 8/50
180/180 [==============================] - 1s 8ms/step - loss: 0.0037 - val_loss: 0.0019
Epoch 9/50
180/180 [==============================] - 1s 8ms/step - loss: 0.0033 - val_loss: 0.0035
Epoch 10/50
180/180 [==============================] - 1s 8ms/step - loss: 0.0028 - val_loss: 0.00

**5. 학습결과 확인 및 임계치 산정**


In [7]:
# Epoch에 따른 loss 시각화 : 학습이 잘 되었음을 확인 가능
plt.plot(history.history["loss"], label="Training Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Train & Validation Loss vs Epoch")
plt.legend()
my_plot(plt)
my_export(np, "training_status_loss", history.history["loss"])
my_export(np, "training_status_val_loss", history.history["val_loss"])

# 정규화 되어 있는 Input, Output을 복원하고, 그 차이의 mean을 구해, Anomaly Score 산출
x_train_unno = (x_train * training_std) + training_mean
x_train_pred = model.predict(x_train)
x_train_pred_unno = (x_train_pred * training_std) + training_mean
train_anomaly_score = np.mean(np.abs((x_train_pred_unno) - (x_train_unno)), axis=1)

# Threshold를 추출하기 위해 학습데이터의 anomaly score를 histogram으로 표현,
# anomaly score가 가장 빈번히 발생한 score영역을 탐색 (정상 데이터 군집)
# peak기준 오른쪽 (anomaly score가 큰 영역에서 hist가 5보다 작은값을 Threhold로 선택)
# 일반적으로는 학습데이터의 max anomaly score를 histogram으로 산정하나, 학습데이터에
# 불량이 포함되어 있을 경우가 있음으로 hist가 5보다 작은곳을 Threhold로 선택함
# 학습데이터가 1년치 데이터임으로, 1년동안 5번 이하로 발생한 센서 데이터이며 anomaly score가 클 경우를
# Threshold로 산정했다는 의미임
thresh = []
hist, bins = np.histogram(train_anomaly_score, 60)
hist = np.append(hist, np.array([0]))
for i in range(0, len(hist)):
    if hist[i] < 5:
        thresh.append(bins[i])
    if max(hist) == hist[i]:
        peak_loc = i

peak = bins[peak_loc]
zero = 0
thresh2 = []
for i in range(0, len(thresh)):
    if thresh[i] > peak:
        thresh2.append(thresh[i])
threshold = min(thresh2)

plt.hist(train_anomaly_score, bins=60)
plt.title("Trained Data - Predicted Data")
plt.xlabel("Train Anomaly Score")
plt.ylabel("No of samples")
my_plot(plt)
my_export(np, "train_anomaly_score", train_anomaly_score)
print(f"Reconstruction threshold value: {threshold:3f}")


Reconstruction threshold value: 0.000138


**7. 시각화**

In [8]:
# 모든 센서데이터를 동시에 PLOT하여 일반적으로 어떻게 sensor data가 움직였는지 확인
for i in range(0, len(x_train_pred_unno)):
    plt.plot(x_train_unno[i], label='Training Data')
    plt.title("1 year Data")
    plt.xlabel("Time")
    plt.ylabel("Sensor Value")
my_plot(plt)
one_year = np.squeeze(x_train_pred_unno)
my_export(np, "one_year", one_year)

# INPUT, OUTPUT을 PLOT하여 학습이 잘 되었는지 확인(5건)
paper_train = x_train_unno[0]
paper_predict = x_train_pred_unno[0]
for i in range(0, 5):
    plt.plot(x_train_unno[i], label='Ground')
    plt.plot(x_train_pred_unno[i], label='Predicted')
    plt.legend()
    plt.title("Normal Data")
    plt.xlabel("Time")
    plt.ylabel("Sensor Value")
    my_plot(plt)
    paper_train = np.hstack((paper_train, x_train_unno[i]))
    paper_predict = np.hstack((paper_predict, x_train_pred_unno[i]))

**8. 불량 판정**

In [9]:
# INPUT데이터를 사용하여 불량 판정하고 INPUT, OUPUT차이 확인
anomalies = train_anomaly_score > threshold
paper_train = np.delete(paper_train, 0, axis=1)
paper_predict = np.delete(paper_predict, 0, axis=1)
my_export(np, "train", paper_train)
my_export(np, "predict", paper_predict)

print("Train Data Anomaly Count")
print("Number of anomaly samples: ", np.sum(anomalies))

alarm_list = []
for i in range(0, len(x_train_pred_unno)):
    if train_anomaly_score[i] > threshold:
        alarm_list.append(train_file_id[i])

with open("./data_output/Paper_99(alarm_list_step).csv", "w") as f:
    for i in range(len(alarm_list)):
        f.write(alarm_list[i] + '\n')

paper_train = x_train_unno[0]
paper_predict = x_train_pred_unno[0]
k = 1
for i in range(0, len(x_train_pred_unno)):
    if train_anomaly_score[i] > threshold:
        plt.subplot(5, 5, k)
        plt.plot(x_train_unno[i], label='Ground')
        plt.plot(x_train_pred_unno[i], label='Predicted')
        plt.title(f"{train_file_id[i]}")
        plt.xlabel("Time")
        plt.ylabel("Sensor Value")
        plt.legend()
        k=k+1
        paper_train = np.hstack((paper_train, x_train_unno[i]))
        paper_predict = np.hstack((paper_predict, x_train_pred_unno[i]))
        if k > 25:
            plt.subplots_adjust(wspace=0.3, hspace=0.8)
            my_plot(plt, bigsize=True)
            k = 1

plt.subplots_adjust(wspace=0.3, hspace=0.8)
my_plot(plt, bigsize=True)

paper_train = np.delete(paper_train, 0, axis=1)
paper_predict = np.delete(paper_predict, 0, axis=1)
my_export(np, "train", paper_train)
my_export(np, "predict", paper_predict)


Train Data Anomaly Count
Number of anomaly samples:  31


**9. 테스트 데이터를 통한 검증**

In [10]:
# TEST 데이터 : 7개의 Sample, 불량 5개 정상 2개
# TEST 데이터를 Train 데이터의 mean, std를 활용하여 정규화
df_test_value = (test_sensor - training_mean) / training_std
x_test = df_test_value
x_test_unnor = df_test_value * training_std + training_mean

#  - TEST데이터를 활용하여 예측, 정규화 했던 결과를 복원
x_test_pred = model.predict(df_test_value)
x_test_pred_unnor = x_test_pred * training_std + training_mean

#  - TEST 데이터의 Anomaly Score 산출
paper_test = x_test_unnor[0]
paper_test_predict= x_test_pred_unnor[0]
test_anomaly_score = np.mean(np.abs(x_test_pred_unnor - x_test_unnor), axis=1)
test_anomaly_score = test_anomaly_score.reshape((-1))

#  - TEST 데이터의 Anomaly 산출
anomalies = test_anomaly_score > threshold
paper_test = np.delete(paper_test, 0, axis=1)
paper_test_predict = np.delete(paper_test_predict, 0, axis=1)
my_export(np, "test_abnormal", paper_test)
my_export(np, "test_predict_abnormal", paper_test_predict)

#  - TEST 데이터의 Anomaly 표기
for i in range(0, len(x_test)):
    plt.plot(x_test_unnor[i], label='Ground')
    plt.plot(x_test_pred_unnor[i], label='Predicted ')
    if anomalies[i]==True:
        plt.title("Test Data : Anomaly", color='red', fontweight='bold')
    else:
        plt.title("Test Data : Normal", color='green', fontweight='bold')
    plt.xlabel("Time")
    plt.ylabel("Sensor Value")
    plt.legend()
    my_plot(plt)
    paper_test = np.hstack((paper_test, x_test_unnor[i]))
    paper_test_predict = np.hstack((paper_test_predict, x_test_pred_unnor[i]))

print("Test Data Anomaly Count")
print("Number of anomaly samples: ", np.sum(anomalies))

Test Data Anomaly Count
Number of anomaly samples:  5


In [ ]:
# TEST 데이터의 Anomaly Score histogram plot
# Threshold 기준 왼쪽에 데이터 2개 (정상), 오른쪽 데이터 5개 (불량) 존재 확인
plt.hist(test_anomaly_score, bins=60)
plt.title("TEST Anomaly Score")
plt.xlabel("Anomaly Score")
plt.ylabel("No of samples")
my_plot(plt)
my_export(np, "test_anomaly_score", test_anomaly_score)